# Task Visualisation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
blk, blu, red, grn, ylw, gry, pur, orn, azu, brw = range(10)

COLOURS = [
    "#000",
    "#0074D9", # blue
    "#FF4136", # red
    "#2ECC40", # green
    "#FFDC00", # yellow
    "#AAAAAA", # grey
    "#F012BE", # fuschia
    "#FF851B", # orange
    "#7FDBFF", # teal
    "#870C25"  # brown
]
cmap = colors.ListedColormap(COLOURS)
norm = colors.Normalize(vmin=0, vmax=9)

task = "/kaggle/input/abstraction-and-reasoning-challenge/training/0a938d79.json"
with open(task, 'r') as f:
    data = json.load(f)
    
first_task = data["train"][0]

first_task_in = np.array(first_task["input"], dtype=np.uint8)
plt.imshow(first_task_in, cmap=cmap, norm=norm)
plt.show()
first_task_out = np.array(first_task["output"], dtype=np.uint8)
plt.imshow(first_task_out, cmap=cmap, norm=norm)
plt.show()

Now that we have shown the task let us discuss the DFSA connections.

# DFSA Connection

This notebook will attempt to explore how tasks (a very easy one initially) can be solved by a Deterministic Finite State Automaton (DFSA).  As a primer let me define a DFSA.

A DFSA consists of:
* a finite set of states $Q$
* a finite alphabet $Σ$
* a transition function $δ: Q \times Σ \rightarrow Q$
* an initial state $q_0 \in Q$
* a set of accepting states $F \subseteq Q$

Given that Cellular Automata (CA) have a start state and several rules that they follow in my opinion the connection to DFSA is somewhat natural.  This is because the rules can be the transition function, the start state is trivial and if we take into consideration non-oscillating CA then there is a terminating state as well.


# DFSA

Below I have defined a DFSA where $q_0$ is the input of the task demonstration and there only one accepting state the output.  Both are padded with 0s with shape (1,1) so we can iterate over the working space of the task without many checks.  The alphabet is, in this case, either a neighbourhood of 3x3 around a cell or 1x3.  The reason for the latter shape is used for the symmetry aspect of the task used here.

> Note that currently this only works for the horizontal demonstration not the vertical

Finally, the transition functions consists of the rules, hardcoded here, and there no intermediate states, just a the starting state being updated until completion.

In [ ]:
class DFSA():

    def __init__(self, state_in, state_out, transitions):
        # Initiliase start, end states and pad
        self.start_state = np.pad(state_in, (1,1), 'constant', constant_values=(0,))
        self.end_state = np.pad(state_out, (1,1), 'constant', constant_values=(0,))
        self.transitions = transitions

    def run(self):
        
        current_state = self.start_state.copy()
        
        # Symmetry: handles situation where we need to propagate the colours horizontally
        symmetry_red = np.zeros((1, 3), dtype=np.uint8)
        symmetry_azu = np.zeros((1, 3), dtype=np.uint8)
        symmetry_red[0, 0] = red
        symmetry_azu[0, 0] = azu
        next_symmetry_red = symmetry_red.copy()
        next_symmetry_azu = symmetry_azu.copy()

        next_symmetry_red[0, 2] = azu
        next_symmetry_azu[0, 2] = red
        
        
        while not np.array_equal(current_state, self.end_state):
            # Have we filled a the height of the matrix with this colour?
            if current_state[1, 8] == azu and not ''.join([str(i) for i in symmetry_red.flatten()]) in self.transitions.keys(): # if azu add symmetry
                # if yes then add symmetry transitions
                self.transitions[''.join([str(i) for i in symmetry_red.flatten()])] = next_symmetry_red
                self.transitions[''.join([str(i) for i in symmetry_azu.flatten()])] = next_symmetry_azu
            
            # Go over matrix and keep updating neighbourhoods
            for i in range(1, current_state.shape[0]-1):
                for j in range(1, current_state.shape[1]-2):
                    current_nbh = current_state[i-1:i+2, j-1:j+2]
                    if len(self.transitions.keys()) == 4:
                        current_nbh = current_nbh[1]
                        
                    if ''.join([str(i) for i in current_nbh.flatten()]) not in self.transitions.keys():
                        continue
                        
                    new_nbh = self.transitions[''.join([str(i) for i in current_nbh.flatten()])]
                    if len(self.transitions.keys()) < 4:
                        current_state[i-1:i+2, j-1:j+2] = new_nbh
                    else:
                        current_state[i, j-1:j+2] = new_nbh
        self.evolved_state = current_state
        
    def get_state(self):
        return self.evolved_state

To fill up the columns with the corresponding colours we define two transitions:
* if the current cell is black and above there is a red cell then fill with red
* if the current cell is black and below there is an azure cell then fill with azure

To add these the 3x3 neighbourhood is flattened and a string representation is obtained to act as a key to a dictionary mapping neighbourhood seen to the resulting one.

In [ ]:
transitions = {}
# Vertical
symbol_red = np.zeros((3, 3), dtype=np.uint8)
symbol_azu = np.zeros((3, 3), dtype=np.uint8)
symbol_red[0, 1] = red
symbol_azu[2, 1] = azu

next_red = symbol_red.copy()
next_azu = symbol_azu.copy()
next_red[1, 1] = red
next_azu[1, 1] = azu

transitions[''.join([str(i) for i in symbol_red.flatten()])] = next_red
transitions[''.join([str(i) for i in symbol_azu.flatten()])] = next_azu

dfsa = DFSA(first_task_in, first_task_out, transitions)

dfsa.run()

plt.imshow(dfsa.get_state(), cmap=cmap, norm=norm)
plt.show()

# Closing Remarks

As we can see DFSAs are somewhat natural to solve CA problems in my opinion.  This is the reason I don't believe classical CNNs trained on raw data will work in this case. I would expect that an algorithm utilising DFSA or Pushdown Automata (PDA), which support counting, are better suited to this challenge.

Again this is my opinion coming from solving a single challenge with a hardcoded approach.  However some interesting questions arise which can be investigated.  For example, the transitions rules representing "neighbourhoods" can probably be learned.  Another direction could be to learn DFSAs using AI techniques?

To close I believe a combination of Deep Learning techniques and DFSA might be suited to this challenge.  Given I am no expert feel free to provide some feedback!